# Introduction to Antidromic Stimulation

## The Problem: Classifying Neurons by Their Projection Targets

Understanding brain function requires knowing not just *where* neurons are located, but *where they send their outputs*. In motor cortex, for example, some neurons project directly to the spinal cord to control movement (pyramidal tract neurons, PTNs), while others project to the basal ganglia to coordinate motor planning (corticostriatal neurons, CSNs). These cell types have distinct physiological properties and are differentially affected by neurological diseases like Parkinson's disease.

But here's the challenge: when you insert a recording electrode into motor cortex, you can only "see" the neuron's cell body and nearby processes. How do you determine where that neuron's axon goes, potentially tens of millimeters away?

Several methods exist for neuron classification:
- **Anatomical tracing**: Inject tracers at target sites; neurons that project there become labeled
- **Optogenetics**: Express light-sensitive proteins in specific projection classes
- **Antidromic stimulation**: Stimulate at potential target sites and look for characteristic responses

This tutorial focuses on **antidromic stimulation**, the classical electrophysiological approach used extensively in primate neurophysiology.

## Orthodromic vs Antidromic: The Direction of Action Potentials

Under normal physiological conditions, action potentials travel **orthodromically**, from the cell body (soma), down the axon, to the synaptic terminals at distant targets. This is the "forward" direction that carries information from the neuron to its downstream partners.

However, axons can conduct action potentials in either direction. If you artificially initiate an action potential at the axon terminal (by electrical stimulation), it will travel **antidromically**, backwards up the axon toward the cell body. When this antidromic spike reaches the soma, it can be detected by a recording electrode positioned there.

```
Orthodromic (normal physiology):     Cell Body ---> Axon ---> Target
Antidromic (experimental):           Cell Body <--- Axon <--- [STIM]
```

This bidirectionality is the key insight: if you stimulate at a particular brain region and a recorded neuron responds antidromically, you know that neuron's axon passes through (or terminates in) that region.

## A Specific Example: The Turner Lab Parkinson's Disease Study

To illustrate antidromic stimulation with real data, we will use recordings from a study by Pasquereau & Turner (2011, *Cerebral Cortex* 21:1362-1378). This research used antidromic identification to compare the activity of pyramidal tract neurons (PTNs) and corticostriatal neurons (CSNs) in monkeys before and after inducing parkinsonism with MPTP. By separating these cell types, the researchers discovered that Parkinson's disease pathology selectively affects PTNs while relatively sparing CSNs, a finding that would have been impossible without projection-based classification.

## Anatomical Context: M1, Cerebral Peduncle, and Striatum

In the Pasquereau & Turner study, recordings were made from **primary motor cortex (M1)**, the region of cortex most directly involved in generating voluntary movement commands. M1 contains multiple types of projection neurons, but two major classes were of interest:

### Pyramidal Tract Neurons (PTNs)
- **Location in cortex**: Layer 5b of M1 (deep layer, large pyramidal cells)
- **Projection target**: Spinal cord, via the corticospinal (pyramidal) tract
- **Axon pathway**: Axons descend through the corona radiata, enter the internal capsule, and converge in the **cerebral peduncle**, a massive white matter bundle at the base of the midbrain that carries all descending cortical motor commands
- **Function**: Direct control of spinal motor neurons and interneurons; execution of fine, skilled movements
- **Stimulation site for identification**: Cerebral peduncle (where PTN axons are densely bundled together)

### Corticostriatal Neurons (CSNs)
- **Location in cortex**: Layer 5 of M1 (intratelencephalic type)
- **Projection target**: Striatum (specifically the putamen, the motor division of the basal ganglia)
- **Axon pathway**: Axons project to the ipsilateral **posterolateral putamen**, which receives topographically organized inputs from motor cortices
- **Function**: Provide cortical information to basal ganglia circuits involved in action selection, motor learning, and movement vigor
- **Stimulation site for identification**: Posterolateral putamen (striatum)

By placing stimulating electrodes in both the cerebral peduncle and the putamen, researchers can determine which neurons are PTNs (respond to peduncle stimulation), which are CSNs (respond to putamen stimulation), and which are neither or both.

## Let's See an Antidromic Response

Before diving into technical details, let's look at what an antidromic response actually looks like. The data from the Pasquereau & Turner study has been publicly released on [DANDI Archive (Dandiset 001636)](https://dandiarchive.org/dandiset/001636/draft). We'll stream it directly and plot a single sweep from an antidromic stimulation experiment.

The session we'll use contains a corticostriatal neuron (CSN) that was identified by antidromic stimulation from the striatum.

In [ ]:
import h5py
import remfile
import numpy as np
import matplotlib.pyplot as plt
from pynwb import NWBHDF5IO
from dandi.dandiapi import DandiAPIClient

from notebook_helpers import get_sweep_waveforms, measure_response_latency, measure_response_amplitude

# Connect to DANDI and get the dandiset
dandiset_id = "001636"
client = DandiAPIClient()
dandiset = client.get_dandiset(dandiset_id, "draft")

# Select a session with a corticostriatal neuron
asset_path = "sub-V/sub-V_ses-V++v4806++PostMPTP++Depth17700um++20000204_behavior+ecephys.nwb"
assets = list(dandiset.get_assets())
asset = next(a for a in assets if a.path == asset_path)
print(f"Streaming: {asset.path}")

# Stream the NWB file directly from DANDI (no download required)
s3_url = asset.get_content_url(follow_redirects=1, strip_query=False)
file_system = remfile.File(s3_url)
file = h5py.File(file_system, mode="r")

io = NWBHDF5IO(file=file)
nwbfile = io.read()
print(f"Session: {nwbfile.session_id}")

In [ ]:
# Get the antidromic sweeps and neuron data
units_df = nwbfile.units.to_dataframe()
sweeps_df = nwbfile.intervals["AntidromicSweepsIntervals"].to_dataframe()
antidromic_latency = units_df['antidromic_latency_ms'].iloc[0]

# Plot a single sweep
sweep = sweeps_df.iloc[0]
time_ms, response_uv, stim_ua = get_sweep_waveforms(sweep)

fig, axes = plt.subplots(2, 1, figsize=(14, 8), sharex=True)

# Top panel: Stimulation current (cause)
axes[0].plot(time_ms, stim_ua, 'r-', linewidth=1.5)
axes[0].axvline(0, color='red', linestyle='--', linewidth=2)
axes[0].set_ylabel('Stimulation Current (uA)', fontsize=16)
axes[0].set_title('One Antidromic Sweep', fontsize=20, fontweight='bold')
axes[0].tick_params(axis='both', labelsize=14)
axes[0].spines['top'].set_visible(False)
axes[0].spines['right'].set_visible(False)
axes[0].text(-20, axes[0].get_ylim()[1] * 0.8, 'Stimulus in\nStriatum', fontsize=18, fontweight='bold', va='top')

# Bottom panel: Neural response (effect)
axes[1].plot(time_ms, response_uv, 'b-', linewidth=1.5)
axes[1].axvline(0, color='red', linestyle='--', linewidth=2, label='Stimulation')
axes[1].axvline(antidromic_latency, color='green', linestyle=':', linewidth=2, label=f'Antidromic response ({antidromic_latency} ms)')
axes[1].set_xlabel('Time relative to stimulation (ms)', fontsize=16)
axes[1].set_ylabel('Neural Response (uV)', fontsize=16)
axes[1].tick_params(axis='both', labelsize=14)
axes[1].legend(loc='upper right', fontsize=14)
axes[1].spines['top'].set_visible(False)
axes[1].spines['right'].set_visible(False)
axes[1].text(-20, axes[1].get_ylim()[1] * 0.8, 'Recording in\nMotor Cortex', fontsize=18, fontweight='bold', va='top')

plt.tight_layout()
plt.show()

**That's it.** That's an antidromic response. The method is conceptually simple: stimulate at a potential axon target (here, the striatum), record at the cell body (here, motor cortex), and if the neuron's axon passes through the stimulation site, you see a response.

Looking at the plot from top to bottom:

The **top panel** shows the stimulus delivered to the striatum. Notice the **biphasic waveform**, first positive, then negative (or vice versa). This charge-balanced pulse is standard for neural stimulation because it minimizes tissue damage and electrode corrosion. The equal-and-opposite phases ensure no net charge accumulates at the electrode.

The **bottom panel** shows what we recorded in motor cortex. At **t = 0 ms** (red dashed line), there's a large, sharp deflection: the **stimulation artifact**. This is electrical contamination from the stimulus pulse, not neural activity. Then, at **t = 6.4 ms** (green dotted line), there's a smaller but distinct waveform: the **antidromic spike**. This is the action potential that traveled backwards up the axon from the striatum to the motor cortex cell body.

The time between stimulation and response is called the **latency** (here, 6.4 ms). It equals distance divided by conduction velocity, so it reflects both how far the spike must travel and how fast this particular axon conducts. Different neuron types have characteristic latencies:

| Cell Type | Stimulation Site | Typical Latency | Why |
|-----------|-----------------|-----------------|-----|
| **PTN** | Cerebral peduncle | 0.6-2.5 ms | Large, heavily myelinated axons = fast conduction |
| **CSN** | Striatum | 2-20 ms | Smaller axons = slower conduction |

The 6.4 ms latency here is consistent with a corticostriatal neuron (CSN).

## How Antidromic Data is Organized in NWB

Now that we've seen what an antidromic response looks like, let's understand how this data is stored in the NWB files for this dandiset. The antidromic testing data is organized into two main components:

### 1. The Units Table: Classification Results

The `units` table stores the final classification for each neuron. Access it via `nwbfile.units.to_dataframe()`:

In [ ]:
units_df = nwbfile.units.to_dataframe()

# The antidromic-related columns are:
classification_cols = ['neuron_projection_type', 'antidromic_stimulation_sites', 
                       'antidromic_latency_ms', 'antidromic_threshold']
units_df[classification_cols]

### 2. The AntidromicSweepsIntervals Table: Raw Waveforms

The raw data from each stimulation trial ("sweep") is stored in `nwbfile.intervals["AntidromicSweepsIntervals"]`:

In [ ]:
# Access the sweeps table from nwbfile.intervals
sweeps_df = nwbfile.intervals["AntidromicSweepsIntervals"].to_dataframe()

print(f"The sweeps table has {len(sweeps_df)} sweeps")


print(f"\nSweeps by protocol type:")
print(sweeps_df['stimulation_protocol'].value_counts())

# Key columns for understanding the data structure
display_cols = ['unit_name', 'stimulation_protocol', 'sweep_number', 'location']
sweeps_df[display_cols].head()

### 3. Extracting Waveforms from Sweep References

The `response` and `stimulation` columns contain tuples: `(start_index, count, timeseries_object)`. To extract waveform data:

In [ ]:
# Get a single sweep from the table
example_sweep = sweeps_df.iloc[0]

# The 'response' column contains a tuple: (start_index, count, timeseries_object)
response_ref = example_sweep['response']
start_index, count, response_series = response_ref

print("The 'response' column contains a reference to the raw data:")
print(f"  Start index: {start_index}")
print(f"  Sample count: {count}")
print(f"  TimeSeries name: {response_series.name}")
print(f"  Sampling rate: {response_series.rate} Hz")

# Extract the data and convert to microvolts
raw_data = response_series.data[start_index:start_index + count]
data_uv = raw_data.flatten() * response_series.conversion * 1e6

print(f"\nExtracted waveform: {len(data_uv)} samples")
time_span = count / response_series.rate * 1000  # in ms
print(f"Time span: {time_span:.2f} ms")

The helper function `get_sweep_waveforms()` (from `notebook_helpers.py`) handles this extraction and returns time axis, response, and stimulation arrays ready for plotting.

## Why Do We Need Validation?

The sweep above looks convincing; there's clearly a response at a consistent latency after stimulation. But how do we **know** this is an antidromic response?

The problem is that electrical stimulation can activate neurons through multiple pathways:

1. **Antidromic** (what we want): The stimulus directly activates the recorded neuron's axon, and the spike travels backward to the cell body
2. **Synaptic/Orthodromic** (confounding): The stimulus activates other neurons at the stimulation site, which then synaptically excite the recorded neuron through a forward pathway

Both would produce a spike after stimulation. We need rigorous criteria to distinguish them.

## The Three Criteria for Antidromic Activation

The electrophysiology literature has established three standard criteria for confirming antidromic activation (Fuller & Schlag, 1976, *Brain Research* 112:283-298):

| Criterion | Test | Why It Works |
|-----------|------|-------------|
| **Fixed latency** | Response occurs at constant delay across trials | Axon conduction time is invariant; synaptic delays fluctuate |
| **Collision test** | Spontaneous spike blocks the antidromic response | Two action potentials annihilate when they meet on the same axon |
| **Frequency following** | Neuron follows stimulation at >200 Hz | Axons can fire rapidly; synapses fatigue at high frequencies |

The **collision test** is often considered the gold standard because it's the most definitive: it can only happen if the orthodromic and antidromic spikes are traveling on the same axon.

Let's illustrate each criterion with real data.

---

## Criterion 1: Fixed Latency

The first criterion tests whether the response latency is constant across trials. Axon conduction time is determined by physical properties (axon length and conduction velocity) that don't change from trial to trial. Synaptic transmission, in contrast, is probabilistic and introduces variable delays.

Let's overlay multiple sweeps to see if the latency is fixed:

In [ ]:
# Get only Collision sweeps for fixed latency demonstration
# (FrequencyFollowing sweeps have 3 pulses per sweep which would confuse the overlay)
all_sweeps = sweeps_df[sweeps_df['stimulation_protocol'] == 'Collision'].copy()

fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Left panel: Overlay all sweeps
ax = axes[0]
for i in range(len(all_sweeps)):
    sweep = all_sweeps.iloc[i]
    time_ms, response_uv, _ = get_sweep_waveforms(sweep)
    ax.plot(time_ms, response_uv, 'b-', linewidth=0.8, alpha=0.4)

ax.axvline(0, color='red', linestyle='--', linewidth=2, label='Stimulation')
ax.axvline(antidromic_latency, color='green', linestyle=':', linewidth=2, 
           label=f'Expected latency ({antidromic_latency} ms)')
ax.set_xlim(-2, 15)
ax.set_xlabel('Time relative to stimulation (ms)', fontsize=16)
ax.set_ylabel('Neural Response (uV)', fontsize=16)
ax.set_title(f'Collision Sweeps Overlaid (n={len(all_sweeps)})', fontsize=18, fontweight='bold')
ax.legend(loc='upper right', fontsize=14)
ax.tick_params(axis='both', labelsize=14)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

# Right panel: Waterfall view showing individual sweeps
ax = axes[1]
offset = 0
spacing = 60  # uV between traces

for i in range(min(15, len(all_sweeps))):
    sweep = all_sweeps.iloc[i]
    time_ms, response_uv, _ = get_sweep_waveforms(sweep)
    ax.plot(time_ms, response_uv + offset, 'b-', linewidth=1.0)
    ax.text(-3, offset, f'#{i+1}', fontsize=10, va='center')
    offset += spacing

ax.axvline(0, color='red', linestyle='--', linewidth=2, alpha=0.7)
ax.axvline(antidromic_latency, color='green', linestyle=':', linewidth=2, alpha=0.7)
ax.set_xlim(-5, 15)
ax.set_xlabel('Time relative to stimulation (ms)', fontsize=16)
ax.set_title('Individual Sweeps (Waterfall View)', fontsize=18, fontweight='bold')
ax.tick_params(axis='x', labelsize=14)
ax.set_yticks([])
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_visible(False)

plt.tight_layout()
plt.show()

### What We See: Evidence for Fixed Latency

**Left panel (Overlay):**
- All sweeps are plotted on top of each other
- The responses form a **tight cluster** at exactly 6.4 ms after stimulation
- If latency were variable, we would see the responses spread out horizontally

**Right panel (Waterfall):**
- Each sweep is plotted separately, offset vertically
- You can trace a vertical line at 6.4 ms and see that **every sweep** has its response aligned
- This consistency is the hallmark of antidromic activation

Let's quantify this by measuring the actual latency in each sweep:

In [ ]:
# Measure latency from all sweeps
measured_latencies = []

for i in range(len(all_sweeps)):
    sweep = all_sweeps.iloc[i]
    time_ms, response_uv, _ = get_sweep_waveforms(sweep)
    lat = measure_response_latency(time_ms, response_uv, antidromic_latency)
    if not np.isnan(lat):
        measured_latencies.append(lat)

# Plot latency histogram
fig, axes = plt.subplots(1, 2, figsize=(16, 5))

# Left: Zoomed view of response window
ax = axes[0]
for i in range(len(all_sweeps)):
    sweep = all_sweeps.iloc[i]
    time_ms, response_uv, _ = get_sweep_waveforms(sweep)
    ax.plot(time_ms, response_uv, 'b-', linewidth=0.6, alpha=0.4)

ax.axvline(antidromic_latency, color='green', linestyle=':', linewidth=2)
ax.axvspan(antidromic_latency - 0.5, antidromic_latency + 0.5, alpha=0.2, color='green',
           label='Expected +/- 0.5 ms')
ax.set_xlim(antidromic_latency - 3, antidromic_latency + 3)
ax.set_xlabel('Time (ms)', fontsize=16)
ax.set_ylabel('Response (uV)', fontsize=16)
ax.set_title('Zoomed View of Response Window', fontsize=18, fontweight='bold')
ax.legend(loc='upper right', fontsize=14)
ax.tick_params(axis='both', labelsize=14)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

# Right: Latency histogram
ax = axes[1]
ax.hist(measured_latencies, bins=15, color='steelblue', edgecolor='black', alpha=0.7)
ax.axvline(antidromic_latency, color='green', linestyle=':', linewidth=2,
           label=f'Expected: {antidromic_latency} ms')
ax.axvline(np.mean(measured_latencies), color='red', linestyle='--', linewidth=2,
           label=f'Measured mean: {np.mean(measured_latencies):.2f} ms')
ax.set_xlabel('Measured Latency (ms)', fontsize=16)
ax.set_ylabel('Count', fontsize=16)
latency_std = np.std(measured_latencies)
ax.set_title(f'Latency Distribution (SD = {latency_std:.3f} ms)', fontsize=18, fontweight='bold')
ax.legend(loc='upper right', fontsize=14)
ax.tick_params(axis='both', labelsize=14)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

plt.tight_layout()
plt.show()

print(f"Latency statistics from {len(measured_latencies)} sweeps:")
print(f"  Mean:     {np.mean(measured_latencies):.3f} ms")
print(f"  SD:       {np.std(measured_latencies):.3f} ms")
print(f"  Range:    {np.min(measured_latencies):.3f} to {np.max(measured_latencies):.3f} ms")

### Interpreting the Fixed Latency Results

The histogram shows that measured latencies cluster tightly around the expected value:

- **SD < 1 ms**: This low variability is characteristic of antidromic activation
- **Tight, unimodal distribution**: All responses occur at essentially the same time

For comparison:
| Activation Type | Typical Latency SD |
|-----------------|-------------------|
| **Antidromic** | < 0.5 ms |
| **Monosynaptic** | 0.5 - 1.5 ms |
| **Polysynaptic** | 1.5 - 5+ ms |

**Conclusion**: The fixed latency criterion is satisfied.

---

## Criterion 2: The Collision Test (Gold Standard)

The collision test is the most definitive verification of antidromic activation. It exploits a fundamental property of axons: **two action potentials traveling in opposite directions on the same axon will collide and annihilate each other**.

### How Collision Works

```
Normal antidromic response (no collision):

    MOTOR CORTEX                                         STRIATUM/PUTAMEN
    [Cell Body] ==================== AXON ==================== [STIM]
         |                                                       |
         |                     <---- antidromic spike ----       |
         V                                                       
    Spike recorded                                              


Collision (spontaneous spike blocks response):

    MOTOR CORTEX                                         STRIATUM/PUTAMEN
    [Cell Body] ==================== AXON ==================== [STIM]
         |                                                       |
         |       ---- spontaneous ---->    <---- antidromic ---- |
         |                               X                          
         V                        COLLISION                      
    No spike recorded                                           
```

### The Critical Collision Window

For collision to occur, the spontaneous spike must be traveling down the axon when the antidromic spike is traveling up. This defines a **critical window** before stimulation:

```
Critical window = Antidromic latency + Axonal refractory period
```

If a spontaneous spike occurs within this window before stimulation, the antidromic response should be blocked.

### A Better Example for Collision

To clearly demonstrate collision, we'll load a different session that shows more dramatic amplitude variability between blocked and unblocked responses:

In [ ]:
# Load a different session with clearer collision effects
collision_asset_path = "sub-V/sub-V_ses-V++v4518++PostMPTP++Depth19900um++20000124_behavior+ecephys.nwb"
collision_asset = next(a for a in assets if a.path == collision_asset_path)
print(f"Loading collision example: {collision_asset.path.split('/')[-1]}")

collision_s3_url = collision_asset.get_content_url(follow_redirects=1, strip_query=False)
collision_file = h5py.File(remfile.File(collision_s3_url), mode="r")
collision_io = NWBHDF5IO(file=collision_file)
collision_nwbfile = collision_io.read()

# Get data from this session
collision_units_df = collision_nwbfile.units.to_dataframe()
collision_sweeps_df = collision_nwbfile.intervals["AntidromicSweepsIntervals"].to_dataframe()

collision_latency = collision_units_df['antidromic_latency_ms'].iloc[0]
print(f"Neuron type: {collision_units_df['neuron_projection_type'].iloc[0]}")
print(f"Antidromic latency: {collision_latency} ms")

# Get collision sweeps
collision_sweeps = collision_sweeps_df[collision_sweeps_df['stimulation_protocol'] == 'Collision']
print(f"Collision sweeps: {len(collision_sweeps)}")

# Calculate critical window
refractory_period = 2.0  # ms
critical_window = collision_latency + refractory_period
print(f"Critical collision window: {critical_window:.1f} ms")

# Measure response amplitude for each sweep
amplitudes = [measure_response_amplitude(*get_sweep_waveforms(collision_sweeps.iloc[i])[:2], collision_latency) 
              for i in range(len(collision_sweeps))]
median_amp = np.median(amplitudes)

fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Left panel: All sweeps with critical window
ax = axes[0]
ax.axvspan(-critical_window, 0, alpha=0.2, color='orange', label=f'Critical window ({critical_window:.1f} ms)')

for i in range(len(collision_sweeps)):
    sweep = collision_sweeps.iloc[i]
    time_ms, response_uv, _ = get_sweep_waveforms(sweep)
    ax.plot(time_ms, response_uv, 'b-', linewidth=0.6, alpha=0.3)

ax.axvline(0, color='red', linestyle='--', linewidth=2, label='Stimulation')
ax.axvline(collision_latency, color='green', linestyle=':', linewidth=2, 
           label=f'Expected response ({collision_latency} ms)')
ax.set_xlim(-15, 15)
ax.set_xlabel('Time relative to stimulation (ms)', fontsize=16)
ax.set_ylabel('Neural Response (uV)', fontsize=16)
ax.set_title(f'All Collision Sweeps (n={len(collision_sweeps)})', fontsize=18, fontweight='bold')
ax.legend(loc='upper right', fontsize=12)
ax.tick_params(axis='both', labelsize=14)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

# Right panel: Response amplitude distribution
ax = axes[1]
ax.hist(amplitudes, bins=12, color='steelblue', edgecolor='black', alpha=0.7)
ax.axvline(median_amp, color='red', linestyle='--', linewidth=2, 
           label=f'Median: {median_amp:.1f} uV')
ax.set_xlabel('Response Amplitude (uV)', fontsize=16)
ax.set_ylabel('Count', fontsize=16)
ax.set_title('Response Amplitude Distribution', fontsize=18, fontweight='bold')
ax.legend(loc='upper right', fontsize=14)
ax.tick_params(axis='both', labelsize=14)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

plt.tight_layout()
plt.show()

amp_cv = np.std(amplitudes) / np.mean(amplitudes) * 100
print(f"\nResponse amplitude variability: CV = {amp_cv:.1f}%")
print(f"Range: {np.min(amplitudes):.1f} to {np.max(amplitudes):.1f} uV")

### Interpreting the Collision Test Results

**Left panel (All Sweeps):**
- The **orange shaded region** shows the critical collision window before stimulation
- Looking at the responses around the expected latency, some sweeps have **strong deflections** while others appear **weaker or absent**
- This variability is exactly what we expect from a collision test

**Right panel (Amplitude Histogram):**
- The histogram shows a **wide spread** of response amplitudes
- High amplitude = no collision (antidromic spike reached cell body)
- Low amplitude = likely collision (spontaneous spike blocked the response)

Let's separate the sweeps by amplitude to see this more clearly:

In [ ]:
# Separate sweeps by amplitude (above vs below median)
high_amp_indices = [i for i, amp in enumerate(amplitudes) if amp >= median_amp]
low_amp_indices = [i for i, amp in enumerate(amplitudes) if amp < median_amp]

fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Left: High amplitude sweeps (clear responses, no collision)
ax = axes[0]
ax.axvspan(-critical_window, 0, alpha=0.15, color='orange')

for idx in high_amp_indices:
    sweep = collision_sweeps.iloc[idx]
    time_ms, response_uv, _ = get_sweep_waveforms(sweep)
    ax.plot(time_ms, response_uv, 'b-', linewidth=0.8, alpha=0.5)

ax.axvline(0, color='red', linestyle='--', linewidth=2)
ax.axvline(collision_latency, color='green', linestyle=':', linewidth=2)
ax.set_xlim(-15, 15)
ax.set_xlabel('Time (ms)', fontsize=16)
ax.set_ylabel('Response (uV)', fontsize=16)
ax.set_title(f'Clear Responses (n={len(high_amp_indices)}): No Collision', fontsize=18, fontweight='bold')
ax.tick_params(axis='both', labelsize=14)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

ax.text(0.02, 0.98, 'No spontaneous spike\nin critical window\n-> Response present',
        transform=ax.transAxes, fontsize=12, va='top',
        bbox=dict(boxstyle='round', facecolor='lightgreen', alpha=0.8))

# Right: Low amplitude sweeps (potential collisions)
ax = axes[1]
ax.axvspan(-critical_window, 0, alpha=0.15, color='orange')

for idx in low_amp_indices:
    sweep = collision_sweeps.iloc[idx]
    time_ms, response_uv, _ = get_sweep_waveforms(sweep)
    ax.plot(time_ms, response_uv, 'b-', linewidth=0.8, alpha=0.5)

ax.axvline(0, color='red', linestyle='--', linewidth=2)
ax.axvline(collision_latency, color='green', linestyle=':', linewidth=2)
ax.set_xlim(-15, 15)
ax.set_xlabel('Time (ms)', fontsize=16)
ax.set_ylabel('Response (uV)', fontsize=16)
ax.set_title(f'Reduced Responses (n={len(low_amp_indices)}): Potential Collisions', fontsize=18, fontweight='bold')
ax.tick_params(axis='both', labelsize=14)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

ax.text(0.02, 0.98, 'Spontaneous spike in\ncritical window?\n-> Response blocked',
        transform=ax.transAxes, fontsize=12, va='top',
        bbox=dict(boxstyle='round', facecolor='lightyellow', alpha=0.8))

plt.tight_layout()
plt.show()

### The Collision Effect Visualized

**Left panel (Clear Responses):**
- These sweeps have **high amplitude** responses at the expected latency
- No spontaneous spike occurred in the critical window
- The antidromic spike traveled unimpeded to the cell body

**Right panel (Reduced Responses):**
- These sweeps have **low amplitude** or nearly absent responses
- A spontaneous spike likely occurred in the critical window and collided with the antidromic spike

**Why this is definitive proof:**

This can only happen if the orthodromic and antidromic spikes are on the **same axon**. If the neuron were being activated synaptically, spontaneous spikes would NOT block the response because they travel on different paths.

**Conclusion**: The collision test criterion is satisfied.

---

## Criterion 3: Frequency Following

The third criterion tests whether the neuron can follow high-frequency stimulation. This exploits a key difference between axonal conduction and synaptic transmission:

| Property | Axons (Antidromic) | Synapses (Orthodromic) |
|----------|-------------------|------------------------|
| **Refractory period** | 1-2 ms | N/A |
| **Max following rate** | 300-500 Hz | 50-100 Hz |
| **Fatigue** | None | Rapid (vesicle depletion) |

**Why synapses fail at high frequencies:**
- Synaptic vesicles deplete faster than they can be recycled
- Neurotransmitter receptors desensitize with repeated activation
- Postsynaptic potentials summate and plateau

**Why axons succeed:**
- Action potential propagation is a regenerative process
- No "resources" to deplete
- Only limitation is the refractory period (~1-2 ms for large axons)

If a neuron can reliably respond to stimulation at >200 Hz, the activation must be antidromic.

### The Frequency Following Protocol

In this dataset, "FrequencyFollowing" sweeps deliver **3 biphasic stimulation pulses** at approximately **400 Hz** (~2.5 ms inter-pulse interval) within each 50 ms sweep. The test asks: does the neuron respond to ALL pulses in the train?

Let's load a session with clear frequency following data:

In [ ]:
# Load a session with clear frequency following data
ff_asset_path = "sub-V/sub-V_ses-V++v1601++PreMPTP++Depth17800um++19990517_behavior+ecephys.nwb"
ff_asset = next(a for a in assets if a.path == ff_asset_path)
print(f"Loading frequency following example: {ff_asset.path.split('/')[-1]}")

ff_s3_url = ff_asset.get_content_url(follow_redirects=1, strip_query=False)
ff_file = h5py.File(remfile.File(ff_s3_url), mode="r")
ff_io = NWBHDF5IO(file=ff_file)
ff_nwbfile = ff_io.read()

# Get data from this session
ff_units_df = ff_nwbfile.units.to_dataframe()
ff_sweeps_df = ff_nwbfile.intervals["AntidromicSweepsIntervals"].to_dataframe()

ff_latency = ff_units_df['antidromic_latency_ms'].iloc[0]
print(f"Neuron type: {ff_units_df['neuron_projection_type'].iloc[0]}")
print(f"Antidromic latency: {ff_latency} ms")

# Get frequency following sweeps
ff_sweeps = ff_sweeps_df[ff_sweeps_df['stimulation_protocol'] == 'FrequencyFollowing']
print(f"FrequencyFollowing sweeps: {len(ff_sweeps)}")

In [ ]:
from scipy.signal import find_peaks

def detect_stimulation_pulses(time_ms, stim_ua, threshold_ua=20):
    """Detect individual stimulation pulses within a sweep."""
    peaks, _ = find_peaks(stim_ua, height=threshold_ua, distance=20)
    if len(peaks) == 0:
        return np.array([0.0])
    return time_ms[peaks]

# Analyze first sweep to find stimulation pulse times
first_sweep = ff_sweeps.iloc[0]
time_ms, response_uv, stim_ua = get_sweep_waveforms(first_sweep)
stim_pulse_times = detect_stimulation_pulses(time_ms, stim_ua)
n_pulses = len(stim_pulse_times)

# Calculate timing
inter_pulse_interval = np.mean(np.diff(stim_pulse_times)) if n_pulses > 1 else 0
freq_hz = 1000 / inter_pulse_interval if inter_pulse_interval > 0 else 0
expected_response_times = stim_pulse_times + ff_latency

print(f"Stimulation structure:")
print(f"  Pulses per sweep: {n_pulses}")
print(f"  Pulse times: {stim_pulse_times} ms")
print(f"  Inter-pulse interval: {inter_pulse_interval:.2f} ms ({freq_hz:.0f} Hz)")
print(f"  Expected response times: {expected_response_times} ms")

In [ ]:
# Visualize frequency following
fig, axes = plt.subplots(2, 2, figsize=(16, 10))

# Panel A: All sweeps overlaid with multi-pulse markers
ax = axes[0, 0]
for i in range(len(ff_sweeps)):
    sweep = ff_sweeps.iloc[i]
    t_ms, resp_uv, _ = get_sweep_waveforms(sweep)
    ax.plot(t_ms, resp_uv, 'b-', linewidth=0.6, alpha=0.4)

# Mark each stimulation pulse and expected response
for j, (stim_t, resp_t) in enumerate(zip(stim_pulse_times, expected_response_times)):
    color = ['red', 'orange', 'darkred'][j % 3]
    ax.axvline(stim_t, color=color, linestyle='--', linewidth=1.5, alpha=0.8)
    ax.axvline(resp_t, color='green', linestyle=':', linewidth=1.5, alpha=0.8)

ax.set_xlim(-2, 15)
ax.set_xlabel('Time (ms)', fontsize=16)
ax.set_ylabel('Response (uV)', fontsize=16)
ax.set_title(f'{n_pulses} Pulses at {freq_hz:.0f} Hz (n={len(ff_sweeps)} sweeps)', fontsize=18, fontweight='bold')
ax.tick_params(axis='both', labelsize=14)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

# Add legend
ax.axvline(-100, color='red', linestyle='--', linewidth=2, label='Stimulation pulses')
ax.axvline(-100, color='green', linestyle=':', linewidth=2, label=f'Expected responses (+{ff_latency} ms)')
ax.legend(loc='upper right', fontsize=12)

# Panel B: Waterfall view showing individual sweeps with all responses
ax = axes[0, 1]
offset = 0
spacing = 100

for i in range(min(12, len(ff_sweeps))):
    sweep = ff_sweeps.iloc[i]
    t_ms, resp_uv, _ = get_sweep_waveforms(sweep)
    ax.plot(t_ms, resp_uv + offset, 'b-', linewidth=0.8)
    ax.text(-4, offset, f'#{i+1}', fontsize=10, va='center')
    offset += spacing

for stim_t, resp_t in zip(stim_pulse_times, expected_response_times):
    ax.axvline(stim_t, color='red', linestyle='--', linewidth=1, alpha=0.6)
    ax.axvline(resp_t, color='green', linestyle=':', linewidth=1, alpha=0.6)

ax.set_xlim(-5, 15)
ax.set_xlabel('Time (ms)', fontsize=16)
ax.set_title('Waterfall: Multiple Responses per Sweep', fontsize=18, fontweight='bold')
ax.tick_params(axis='x', labelsize=14)
ax.set_yticks([])
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_visible(False)

# Panel C: Mean response showing multi-pulse structure
ax = axes[1, 0]
all_responses = []
for i in range(len(ff_sweeps)):
    sweep = ff_sweeps.iloc[i]
    t_ms, resp_uv, _ = get_sweep_waveforms(sweep)
    all_responses.append(resp_uv)

all_responses = np.array(all_responses)
mean_response = np.mean(all_responses, axis=0)
std_response = np.std(all_responses, axis=0)

ax.fill_between(t_ms, mean_response - std_response, mean_response + std_response,
                alpha=0.3, color='blue', label='SD')
ax.plot(t_ms, mean_response, 'b-', linewidth=2, label='Mean response')

for stim_t, resp_t in zip(stim_pulse_times, expected_response_times):
    ax.axvline(stim_t, color='red', linestyle='--', linewidth=1.5, alpha=0.7)
    ax.axvline(resp_t, color='green', linestyle=':', linewidth=1.5, alpha=0.7)

ax.set_xlim(-2, 15)
ax.set_xlabel('Time (ms)', fontsize=16)
ax.set_ylabel('Response (uV)', fontsize=16)
ax.set_title('Mean Response: Clear Following of All Pulses', fontsize=18, fontweight='bold')
ax.legend(loc='upper right', fontsize=12)
ax.tick_params(axis='both', labelsize=14)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

# Panel D: Response amplitude by pulse number
ax = axes[1, 1]

# Measure amplitude for each pulse across all sweeps
pulse_amplitudes = {j: [] for j in range(n_pulses)}
for i in range(len(ff_sweeps)):
    sweep = ff_sweeps.iloc[i]
    t_ms, resp_uv, _ = get_sweep_waveforms(sweep)
    for j, resp_t in enumerate(expected_response_times):
        amp = measure_response_amplitude(t_ms, resp_uv, resp_t, window_ms=1.0)
        pulse_amplitudes[j].append(amp)

# Bar plot with error bars
pulse_nums = np.arange(1, n_pulses + 1)
means = [np.mean(pulse_amplitudes[j]) for j in range(n_pulses)]
stds = [np.std(pulse_amplitudes[j]) for j in range(n_pulses)]
colors_bar = ['steelblue', 'coral', 'seagreen']

bars = ax.bar(pulse_nums, means, yerr=stds, capsize=8, 
              color=[colors_bar[j % 3] for j in range(n_pulses)], 
              edgecolor='black', alpha=0.8)

ax.set_xlabel('Pulse Number', fontsize=16)
ax.set_ylabel('Response Amplitude (uV)', fontsize=16)
ax.set_title('Amplitude Remains Consistent Across Pulses', fontsize=18, fontweight='bold')
ax.set_xticks(pulse_nums)
ax.tick_params(axis='both', labelsize=14)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

plt.tight_layout()
plt.show()

# Print summary
print(f"\nFrequency Following Summary:")
print(f"  Stimulation frequency: {freq_hz:.0f} Hz")
for j in range(n_pulses):
    print(f"  Pulse {j+1}: {means[j]:.1f} +/- {stds[j]:.1f} uV")

### Interpreting the Frequency Following Results

**Panel A (All Sweeps):**
- Three stimulation pulses (red/orange dashed lines) delivered at ~400 Hz
- Green dotted lines show expected response times (stimulus time + latency)
- Responses cluster at all three expected times

**Panel B (Waterfall):**
- Each sweep shows THREE distinct responses, one after each stimulus
- The neuron reliably follows all pulses in the train

**Panel C (Mean Response):**
- Averaging across sweeps reveals clear peaks at each expected response time
- The response waveform is consistent for all three pulses

**Panel D (Amplitude by Pulse):**
- Response amplitude remains relatively consistent across pulses
- Some decrease for later pulses is normal, but responses are still present
- Synaptic activation would show dramatic failure (>80% amplitude drop) by pulse 2-3

**Why this proves antidromic activation:**

At 400 Hz stimulation, each pulse arrives only 2.5 ms after the previous one. Synaptic transmission cannot sustain this rate because:
1. Vesicle pools deplete within 2-3 pulses
2. Postsynaptic receptors desensitize
3. The neurotransmitter reuptake/recycling machinery cannot keep up

The fact that this neuron responds to all three pulses with consistent amplitude demonstrates direct axonal activation.

**Conclusion**: The frequency following criterion is satisfied.

---

## Sessions Used in This Tutorial

We demonstrated the antidromic criteria using three sessions from the same dataset, each chosen for clarity:

| Criterion | Asset Path | Neuron Type | Why This Session |
|-----------|------------|-------------|------------------|
| **Fixed Latency** | `sub-V/sub-V_ses-V++v4806++PostMPTP++Depth17700um++20000204_behavior+ecephys.nwb` | CSN | 31 sweeps with consistent responses |
| **Collision Test** | `sub-V/sub-V_ses-V++v4518++PostMPTP++Depth19900um++20000124_behavior+ecephys.nwb` | PTN | 25 collision sweeps with high amplitude variability |
| **Frequency Following** | `sub-V/sub-V_ses-V++v1601++PreMPTP++Depth17800um++19990517_behavior+ecephys.nwb` | PTN | 31 sweeps with clear multi-pulse structure at 400 Hz |

This illustrates a key advantage of standardized data formats (NWB) and public repositories (DANDI): you can easily access multiple datasets to find the clearest examples.

## PTN vs CSN: Two Projection Types

The neurons we examined were corticostriatal neurons (CSNs). For comparison, here's how PTNs and CSNs differ:

| Property | Pyramidal Tract Neurons (PTNs) | Corticostriatal Neurons (CSNs) |
|----------|-------------------------------|--------------------------------|
| **Projection target** | Spinal cord via cerebral peduncle | Striatum (putamen) |
| **Stimulation site** | Cerebral peduncle | Posterolateral putamen |
| **Antidromic latency** | 0.6-2.5 ms | 2-20 ms |
| **Axon diameter** | Large (fast conduction) | Smaller (slower conduction) |
| **Cortical layer** | Layer 5b (large pyramidal) | Layer 5 (intratelencephalic) |
| **Function** | Direct motor commands to spinal cord | Cortical input to basal ganglia |

In [ ]:
# Display classification for the first neuron we examined
unit = units_df.iloc[0]

print("Example neuron classification (from session v4806):")
print(f"  Type: {unit['neuron_projection_type']}")
print(f"  Stimulation site: {unit['antidromic_stimulation_sites']}")
print(f"  Latency: {unit['antidromic_latency_ms']} ms")

# Estimate conduction velocity (approximate distance M1 to striatum: ~30 mm)
distance_mm = 30
conduction_velocity = distance_mm / unit['antidromic_latency_ms']
print(f"  Estimated conduction velocity: ~{conduction_velocity:.1f} m/s")

**Further reading:**
- Pasquereau B, Turner RS (2011) Primary motor cortex of the parkinsonian monkey. *Cerebral Cortex* 21:1362-1378.
- Fuller JH, Schlag JD (1976) Determination of antidromic excitation by the collision test. *Brain Research* 112:283-298.